In [1]:
# coding=utf-8
import pandas as pd
import csv  # Import csv
import tweepy
import re
import string
import nltk
import pandas as pd
from nltk import FreqDist
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
import numpy as np
from sinling import SinhalaTokenizer, POSTagger
#pd.set_option('display.max_colwidth',1000)
tokenizer = SinhalaTokenizer()

In [2]:
#!/usr/bin/env python
# -*- coding:utf-8 -*-

import codecs
df = pd.read_csv('FinalListPreprocessedData.csv', header = None, encoding='utf-8', engine='python')
df.columns = ['ID','User','CREATED_AT','LABEL','stemming_words','LIKE COUNT','FOLLOWERS COUNT']
df = df.dropna()
df.head()

,ID,User,CREATED_AT,LABEL,stemming_words,LIKE COUNT,FOLLOWERS COUNT
1,2.0,1.42E+18,7/29/2021 2:40,Other,හාමුදුරුවරු සියලූම පූජකයන්ගේ මානසික සෞඛ්යය සොය...,1,1433
2,3.0,1.42E+18,7/27/2021 10:40,Politics,ද්විපාර්ශවික එකඟතා ජන ජීවිත සෞඛ්යය මනාව ආරක්ෂා...,10,13898
3,4.0,1.42E+18,7/25/2021 20:56,Other,කරැනාකර චින නිෂ්පාදිත එන්නත ලබාගත් පුද්ගලයන් හ...,0,87
4,5.0,1.42E+18,7/25/2021 20:53,Other,කරැනාකර චින නිෂ්පාදිත එන්නත ලබාගත් පුද්ගලයන් හ...,0,87
5,6.0,1.42E+18,7/25/2021 20:52,Other,කරැනාකර චින නිෂ්පාදිත එන්නත ලබාගත් පුද්ගලයන් හට,1,87


In [3]:
tokenizer = SinhalaTokenizer()
def clean_text(text):
    tokenized_sentences = [tokenizer.tokenize(f'{ss}.') for ss in tokenizer.split_sentences(text)]
    tagger = POSTagger()
    pos_tags =tagger.predict(tokenized_sentences) 
    return pos_tags 



In [6]:
df['cleaned_msg'] = df['stemming_words'].apply(lambda x: clean_text(x))
df.tail()

,ID,User,CREATED_AT,LABEL,stemming_words,LIKE COUNT,FOLLOWERS COUNT,cleaned_msg
529,1288.0,1.42E+18,7/28/2021 15:30,Other,කොවිඩ් මරණ කොවිඩ් මරණ සෞඛ්ය සේවා අධ්යක්ෂ ජනරාල...,6,110509,"[[(කොවිඩ්, NNP), (මරණ, NNJ), (කොවිඩ්, NNJ), (ම..."
530,1290.0,1.42E+18,7/27/2021 6:51,Crime,ඉෂාලිනිගේ මරණ පරීක්ෂණයට දේශපාලන බලපෑම්,0,704,"[[(ඉෂාලිනිගේ, NNP), (මරණ, NNJ), (පරීක්ෂණයට, NN..."
531,1296.0,1.42E+18,7/27/2021 2:09,Crime,දියණියගේ මරණ පරීක්ෂණය කලේ මුස්ලිම් දොස්තර කෙනෙ...,0,1631,"[[(දියණියගේ, NNC), (මරණ, NNJ), (පරීක්ෂණය, NNC)..."
532,1300.0,1.42E+18,7/27/2021 1:27,Other,පිරිතෙන් ප්රතිපල ලැබීම එය පුද්ගලයෙකු සජ්ජායනය ...,1,317,"[[(පිරිතෙන්, NNC), (ප්රතිපල, NNC), (ලැබීම, VNN..."
533,1302.0,1.42E+18,7/27/2021 1:06,Crime,රිෂාඩ් බදියුදීන් හිටපු අමාත්යවරයාගේ නිවසේ ගිනි...,0,830,"[[(රිෂාඩ්, NNP), (බදියුදීන්, NNP), (හිටපු, VP)..."


In [7]:
def pos_tag(data):
    result = []
    if type(data) == tuple:
        return data[1]

    if type(data) == list:
        for inner in data:
            result.append(pos_tag(inner))
    return result 

In [9]:
df['pos_tagging'] = df['cleaned_msg'].apply(lambda x: pos_tag(x))
df.tail()

,ID,User,CREATED_AT,LABEL,stemming_words,LIKE COUNT,FOLLOWERS COUNT,cleaned_msg,pos_tagging
529,1288.0,1.42E+18,7/28/2021 15:30,Other,කොවිඩ් මරණ කොවිඩ් මරණ සෞඛ්ය සේවා අධ්යක්ෂ ජනරාල...,6,110509,"[[(කොවිඩ්, NNP), (මරණ, NNJ), (කොවිඩ්, NNJ), (ම...","[[NNP, NNJ, NNJ, NNJ, NNJ, NNJ, JJ, NNC, JCV, ..."
530,1290.0,1.42E+18,7/27/2021 6:51,Crime,ඉෂාලිනිගේ මරණ පරීක්ෂණයට දේශපාලන බලපෑම්,0,704,"[[(ඉෂාලිනිගේ, NNP), (මරණ, NNJ), (පරීක්ෂණයට, NN...","[[NNP, NNJ, NNC, JJ, NNC, FS]]"
531,1296.0,1.42E+18,7/27/2021 2:09,Crime,දියණියගේ මරණ පරීක්ෂණය කලේ මුස්ලිම් දොස්තර කෙනෙ...,0,1631,"[[(දියණියගේ, NNC), (මරණ, NNJ), (පරීක්ෂණය, NNC)...","[[NNC, NNJ, NNC, NNC, NNP, NNC, NNC, NNC, VP, ..."
532,1300.0,1.42E+18,7/27/2021 1:27,Other,පිරිතෙන් ප්රතිපල ලැබීම එය පුද්ගලයෙකු සජ්ජායනය ...,1,317,"[[(පිරිතෙන්, NNC), (ප්රතිපල, NNC), (ලැබීම, VNN...","[[NNC, NNC, VNN, PRP, NNC, NCV, VNN, NNC, PRP,..."
533,1302.0,1.42E+18,7/27/2021 1:06,Crime,රිෂාඩ් බදියුදීන් හිටපු අමාත්යවරයාගේ නිවසේ ගිනි...,0,830,"[[(රිෂාඩ්, NNP), (බදියුදීන්, NNP), (හිටපු, VP)...","[[NNP, NNP, VP, NNC, NNC, NNJ, NNC, NNC, VP, P..."


In [10]:
import gensim
from gensim.models import Word2Vec
import nltk
from nltk.corpus import stopwords
import re

# My work

In [11]:
def get_pos_vec(x):
    w2v_model = Word2Vec(min_count=1,window=2,vector_size=2,sample=1e-5,alpha=0.01,min_alpha=0.0007,negative=0,workers=2)
    w2v_model.build_vocab(x, progress_per=1)
    w2v_model.train(x, total_examples=w2v_model.corpus_count, epochs=3, report_delay=1)
    
    for index, key in enumerate(w2v_model.wv.key_to_index):
        vec = w2v_model.wv[key]
    return vec

In [12]:
df['pos_vec'] = df['pos_tagging'].apply(lambda x: get_pos_vec(x))

In [13]:
def get_word_vec(text):
    x = [tokenizer.tokenize(f'{ss}.') for ss in tokenizer.split_sentences(text)]
    w2v_model = Word2Vec(min_count=1,window=2,vector_size=2,sample=1e-5,alpha=0.01,min_alpha=0.0007,negative=0,workers=2)
    w2v_model.build_vocab(x, progress_per=1)
    w2v_model.train(x, total_examples=w2v_model.corpus_count, epochs=3, report_delay=1)
    
    for index, key in enumerate(w2v_model.wv.key_to_index):
        vec = w2v_model.wv[key]
    return vec

In [14]:
df['word_vec'] = df['stemming_words'].apply(lambda x: get_word_vec(x))

In [15]:
df.tail()

,ID,User,CREATED_AT,LABEL,stemming_words,LIKE COUNT,FOLLOWERS COUNT,cleaned_msg,pos_tagging,pos_vec,word_vec
529,1288.0,1.42E+18,7/28/2021 15:30,Other,කොවිඩ් මරණ කොවිඩ් මරණ සෞඛ්ය සේවා අධ්යක්ෂ ජනරාල...,6,110509,"[[(කොවිඩ්, NNP), (මරණ, NNJ), (කොවිඩ්, NNJ), (ම...","[[NNP, NNJ, NNJ, NNJ, NNJ, NNJ, JJ, NNC, JCV, ...","[-0.24300802, -0.09080088]","[0.3655883, 0.2535131]"
530,1290.0,1.42E+18,7/27/2021 6:51,Crime,ඉෂාලිනිගේ මරණ පරීක්ෂණයට දේශපාලන බලපෑම්,0,704,"[[(ඉෂාලිනිගේ, NNP), (මරණ, NNJ), (පරීක්ෂණයට, NN...","[[NNP, NNJ, NNC, JJ, NNC, FS]]","[-0.2507714, -0.18816864]","[0.36902523, -0.07667363]"
531,1296.0,1.42E+18,7/27/2021 2:09,Crime,දියණියගේ මරණ පරීක්ෂණය කලේ මුස්ලිම් දොස්තර කෙනෙ...,0,1631,"[[(දියණියගේ, NNC), (මරණ, NNJ), (පරීක්ෂණය, NNC)...","[[NNC, NNJ, NNC, NNC, NNP, NNC, NNC, NNC, VP, ...","[0.14382899, 0.049593687]","[-0.37557912, -0.046502113]"
532,1300.0,1.42E+18,7/27/2021 1:27,Other,පිරිතෙන් ප්රතිපල ලැබීම එය පුද්ගලයෙකු සජ්ජායනය ...,1,317,"[[(පිරිතෙන්, NNC), (ප්රතිපල, NNC), (ලැබීම, VNN...","[[NNC, NNC, VNN, PRP, NNC, NCV, VNN, NNC, PRP,...","[0.36902523, -0.07667363]","[0.3655883, 0.2535131]"
533,1302.0,1.42E+18,7/27/2021 1:06,Crime,රිෂාඩ් බදියුදීන් හිටපු අමාත්යවරයාගේ නිවසේ ගිනි...,0,830,"[[(රිෂාඩ්, NNP), (බදියුදීන්, NNP), (හිටපු, VP)...","[[NNP, NNP, VP, NNC, NNC, NNJ, NNC, NNC, VP, P...","[0.31754446, -0.1702683]","[-0.044401407, 0.2766831]"


In [16]:
X = df['pos_vec'].to_numpy()
X = X.reshape(-1,1)

In [17]:
X.shape

(532, 1)

In [ ]:
X = np.concatenate(np.concatenate(X, axis = 0),axis = 0).reshape(-1,2)

In [ ]:
y=df['LABEL']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

In [ ]:
X_train.shape, X_test.shape

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
clf = LinearSVC()
clf.fit(X_train,y_train)

In [ ]:
y_pred = clf.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report
from sklearn import metrics

print(metrics.confusion_matrix(y_test,y_pred))

print(metrics.classification_report(y_test,y_pred))



In [21]:
# from gensim.test.utils import common_texts
from gensim.models import Word2Vec

# pos_tag_list = [['DT', 'JJ', 'NN', 'NN', 'VBD', 'IN', 'DT', 'NN'],
#                       ['DT','NN','VBD','IN','DT','NN'],['PRP','VBP','JJ','JJ','NNS']]

word_token_list = [['ඉෂාලිනිගේ', 'මරණ', 'පරීක්ෂණයට', 'දේශපාලන', 'බලපෑම්']]

w2v_model = Word2Vec(min_count=1,
                 window=2,
                 vector_size=3,
                 sample=1e-5, 
                 alpha=0.01, 
                 min_alpha=0.0007, 
                 negative=0,
                 workers=2)
w2v_model.build_vocab(pos_tag_list, progress_per=1)
w2v_model.train(pos_tag_list, total_examples=w2v_model.corpus_count, epochs=3, report_delay=1)

# get the vectors for the Pos_tags from w2v_model
my_dict = dict({})
for index, key in enumerate(w2v_model.wv.key_to_index):
    my_dict[key] = w2v_model.wv[key]

print(my_dict)

{'.': array([-0.01787424,  0.00788101,  0.17011166], dtype=float32), 'බලපෑම්': array([ 0.3003091 , -0.31009832, -0.23722696], dtype=float32), 'දේශපාලන': array([ 0.21529572,  0.2990996 , -0.16718094], dtype=float32), 'පරීක්ෂණයට': array([-0.12544577,  0.24601682, -0.05111575], dtype=float32), 'මරණ': array([-0.15122049,  0.21846835, -0.16200535], dtype=float32), 'ඉෂාලිනිගේ': array([-0.06053392,  0.09588599,  0.03306246], dtype=float32)}


In [ ]:
word_token_list = [['ඉෂාලිනිගේ', 'මරණ', 'පරීක්ෂණයට', 'දේශපාලන', 'බලපෑම්']]